模式识别大作业之商品分类_黄国正_20210529

说明：本笔记为PCbighomework_main_hgz的补充，整合了其中表现最好的文本信息特征TF-IDF和最好的图像信息处理SIFT，并用对应方法进行进一步处理和评估。文本分类和图像分类的大部分方法实现和调参见PCbighomework_main_hgz；采用神经网络的方法见PCbighomework_network_hgz

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#加载所有需要的包
#数据处理
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

#绘图读图
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg 
import seaborn as sns

#学习
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC,SVC
import sklearn.svm as svm
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score,roc_auc_score,average_precision_score
from sklearn import neighbors
from sklearn import datasets
from sklearn.datasets import load_digits,load_boston
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor,VotingClassifier,RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing

#文本处理
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import TfidfVectorizer

#图像处理
import os
import cv2
import glob
import joblib
#执行代码时尽量从前往后执行，避免因重复命名报错
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("Set up complete!")

综合文本和图片信息进行分类改进

In [ ]:
#加载全部数据
totaldata = pd.read_csv("../input/shopee-product-matching/train.csv")

In [ ]:
#TF-IDF模型
#超小测试样本及划分
toydata=totaldata[0:100]
toytrickdata=totaldata.loc[totaldata.label_group.isin(toydata.label_group)]
toydata=toytrickdata
tx_train,tx_test, ty_train, ty_test = train_test_split(toydata,toydata.label_group,test_size=0.3, random_state=0)
#超小测试文本样本及划分
toytextdata=toydata[['title','label_group']]
ttextx_train,ttextx_test, ttexty_train, ttexty_test = train_test_split(toytextdata,toytextdata.label_group,test_size=0.3, random_state=0)
#tiidf特征提取
count_vec = TfidfVectorizer(binary=False, decode_error='ignore', stop_words='english')
response = count_vec.fit_transform(ttextx_train.title) # s must be string
trainfeature = response.toarray()
response = count_vec.transform(ttextx_test.title) # s must be string
testfeature = response.toarray()

In [ ]:
#TF-IDF特征上做SVM预测，为了生成概率预测，没有采用线性SVC
from sklearn import svm
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score,roc_auc_score,average_precision_score
# Set dual=False to speed up training, and it's not needed
clf = svm.SVC(random_state=1,  max_iter=10000,probability=True)
clf.fit(trainfeature, ttexty_train)
print(f"TF-IDF SVM Accuracy: {clf.score(testfeature,ttexty_test) * 100:.3f}%", )
print(f"TF-IDF SVM Precision score: {precision_score(ttexty_test,clf.predict(testfeature) ,average='macro',zero_division=0) * 100:.3f}%", )
print(f"TF-IDF SVM Recall score: {recall_score(ttexty_test,clf.predict(testfeature) ,average='macro',zero_division=0) * 100:.3f}%", )
print(f"TF-IDF SVM F1 score: {f1_score(ttexty_test,clf.predict(testfeature),average='macro',zero_division=0) * 100:.3f}%", )



In [ ]:
#图像数据
#超小测试图像样本及划分
toyimagedata=toydata[['image','label_group']]
timagex_train,timagex_test, timagey_train, timagey_test = train_test_split(toyimagedata,toyimagedata.label_group,test_size=0.3, random_state=0)

In [ ]:
#SIFT特征提取，时间较长
def calcSiftFeature(img):
    #设置图像sift特征关键点最大为200
    sift = cv2.SIFT_create()
    #计算图片的特征点和特征点描述
    keypoints, features = sift.detectAndCompute(img, None)
    return features

#计算词袋
def learnVocabulary(features):
    wordCnt = 50
    #criteria表示迭代停止的模式   eps---精度0.1，max_iter---满足超过最大迭代次数20
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.1)
    #得到k-means聚类的初始中心点
    flags = cv2.KMEANS_RANDOM_CENTERS
    # 标签，中心 = kmeans(输入数据（特征)、聚类的个数K,预设标签，聚类停止条件、重复聚类次数、初始聚类中心点
    compactness, labels, centers = cv2.kmeans(features, wordCnt, None,criteria, 20, flags)
    return centers

def calcFeatVec(features, centers):
    featVec = np.zeros((1, 50))
    for i in range(0, features.shape[0]):
        #第i张图片的特征点
        fi = features[i]
        diffMat = np.tile(fi, (50, 1)) - centers
        #axis=1按行求和，即求特征到每个中心点的距离
        sqSum = (diffMat**2).sum(axis=1)
        dist = sqSum**0.5
        #升序排序
        sortedIndices = dist.argsort()
        #取出最小的距离，即找到最近的中心点
        idx = sortedIndices[0]
        #该中心点对应+1
        featVec[0][idx] += 1
    return featVec

features = np.float32([]).reshape(0, 128)#存放训练集图片的特征
for i in range(len(toyimagedata)):
    img = cv2.imread('../input/shopee-product-matching/train_images/'+toyimagedata.image[toyimagedata.index[i]])
    #获取图片sift特征点
    img_f = calcSiftFeature(img)
    #特征点加入训练数据
    features = np.append(features, img_f, axis=0)
#训练集的词袋
centers = learnVocabulary(features)

data_vec = np.float32([]).reshape(0, 50)#存放训练集图片的特征
labels = np.float32([])
for i in range(len(toyimagedata)):
    img = cv2.imread('../input/shopee-product-matching/train_images/'+toyimagedata.image[toyimagedata.index[i]])
    img_f = calcSiftFeature(img)
    img_vec = calcFeatVec(img_f, centers)
    data_vec = np.append(data_vec,img_vec,axis=0)
    labels = np.append(labels,toyimagedata.label_group[toyimagedata.index[i]])



In [ ]:
# Normalization,可调整
tt=data_vec
data_vec = preprocessing.normalize(tt, norm='l2')

In [ ]:
#sift数据划分
tsiftdatax_train, tsiftdatax_test, tsiftdatay_train, tsiftdatay_test = train_test_split(data_vec, labels,
                                                    test_size=0.3, random_state=0)

In [ ]:
#sift上做SVM预测
clf2 = svm.SVC(random_state=1, max_iter=10000,probability=True)
clf2.fit(tsiftdatax_train, tsiftdatay_train)
print(f"SIFT SVM Accuracy: {accuracy_score(tsiftdatay_test,clf2.predict(tsiftdatax_test)) * 100:.3f}%", )
print(f"SIFT SVM Precision score: {precision_score(tsiftdatay_test,clf2.predict(tsiftdatax_test) ,average='macro',zero_division=0) * 100:.3f}%", )
print(f"SIFT SVM Recall score: {recall_score(tsiftdatay_test,clf2.predict(tsiftdatax_test) ,average='macro',zero_division=0) * 100:.3f}%", )
print(f"SIFT SVM F1 score: {f1_score(tsiftdatay_test,clf2.predict(tsiftdatax_test),average='macro',zero_division=0) * 100:.3f}%", )


现在我们已有基于TF-IDF的SVM模型和基于SIFT的SVM模型，我们想利用其概率预测的并集生成新的判断模型，在这之前我们先用SIFT进行实验

In [ ]:
label=np.unique(tsiftdatay_train)

probsift = clf2.predict_proba(tsiftdatax_test)
#用概率生成预测
idxsift = np.argmax(probsift, axis=1)
prosift = np.amax(probsift, axis=1)
predictsift = label[idxsift]
print(f"SIFT SVM Recall score: {recall_score(tsiftdatay_test,predictsift ,average='macro',zero_division=0) * 100:.3f}%", )
#类别不连续，predict_prob最高的标签不一定是predict标签
sum(predictsift==clf2.predict(tsiftdatax_test))

发现用模型生成的概率来推断的标签和直接由模型给出的标签并不相同，这可能是标签不连续导致出现了未曾训练过的新标签造成的，因此之后取并集时需要注意这一点进行灵活预测

下面我们在不同的方法上对TF-IDF和SIFT的分类模型进行整合；通常选取两模型中预测概率更高的标签，但如果出现概率选择与预测标签不一致的情况，我们直接选择TF-IDF的标签。对SVM、KNN、决策树决策森林、Adaboost方法我们都可以取并集并给出结果。

In [ ]:
#TF-IDF与SIFT SVM并集
label=np.unique(tsiftdatay_train)

probtf = clf.predict_proba(testfeature)
#用概率生成预测
idxtf = np.argmax(probtf, axis=1)
protf = np.amax(probtf, axis=1)


probsift = clf2.predict_proba(tsiftdatax_test)
#用概率生成预测
idxsift = np.argmax(probsift, axis=1)
prosift = np.amax(probsift, axis=1)
predictsift = label[idxsift]

#结合生成新的预测，考虑到predict_prob不如predict效果好，不同时我们只采用TF-IDF模型的predict值
predictcombine=np.zeros(idxsift.shape[0])
for i in range(idxsift.shape[0]):
    #出现不一致，只考虑TF-IDF
    if(label[idxtf[i]]!=clf.predict(testfeature)[i] or label[idxsift[i]]!=clf2.predict(tsiftdatax_test)[i]):
        predictcombine[i]=clf.predict(testfeature)[i]
    else:
        if(protf[i]>prosift[i]):
            predictcombine[i]=label[idxtf[i]]
        else:
            predictcombine[i]=label[idxsift[i]]
#打印准确率，测试集相同
print(f"Combining TF-IDF&SIFT SVM Accuracy score: {accuracy_score(tsiftdatay_test,predictcombine) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT SVM Precision score: {precision_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT SVM Recall score: {recall_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT SVM F1 score: {f1_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )

In [ ]:
#KNN分析合并

#TF-IDF特征上做knn分析
from sklearn import neighbors
from sklearn import datasets
#参数k可训练
for k in [3,4,5,6,7,8,9,10,11,12]:
    print("k="+str(k))
    clf = neighbors.KNeighborsClassifier(n_neighbors=k)
    clf.fit(trainfeature, ttexty_train)
    clf2 = neighbors.KNeighborsClassifier(n_neighbors=k)
    clf2.fit(tsiftdatax_train, tsiftdatay_train)
    label=np.unique(tsiftdatay_train)

    probtf = clf.predict_proba(testfeature)
    #用概率生成预测
    idxtf = np.argmax(probtf, axis=1)
    protf = np.amax(probtf, axis=1)


    probsift = clf2.predict_proba(tsiftdatax_test)
    #用概率生成预测
    idxsift = np.argmax(probsift, axis=1)
    prosift = np.amax(probsift, axis=1)
    predictsift = label[idxsift]

    #结合生成新的预测，考虑到predict_prob不如predict效果好，不同时我们只采用TF-IDF模型的predict值
    predictcombine=np.zeros(idxsift.shape[0])
    for i in range(idxsift.shape[0]):
        #出现不一致，只考虑TF-IDF
        if(label[idxtf[i]]!=clf.predict(testfeature)[i] or label[idxsift[i]]!=clf2.predict(tsiftdatax_test)[i]):
            predictcombine[i]=clf.predict(testfeature)[i]
        else:
            if(protf[i]>prosift[i]):
                predictcombine[i]=label[idxtf[i]]
            else:
                predictcombine[i]=label[idxsift[i]]
    #打印准确率，测试集相同
    print(f"Combining TF-IDF&SIFT KNN Accuracy score: {accuracy_score(tsiftdatay_test,predictcombine) * 100:.3f}%", )
    print(f"Combining TF-IDF&SIFT KNN Precision score: {precision_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
    print(f"Combining TF-IDF&SIFT KNN Recall score: {recall_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
    print(f"Combining TF-IDF&SIFT KNN F1 score: {f1_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )

此时KNN选取参数k=3效果最好

In [ ]:
#决策树和随机森林合并
#TF-IDF
dt = DecisionTreeClassifier(random_state=1)
dt.fit(trainfeature, ttexty_train)
rf = RandomForestClassifier(random_state=1)
rf.fit(trainfeature, ttexty_train)
#基于SIFT决策树和随机森林
dt2 = DecisionTreeClassifier(random_state=1)
dt2.fit(tsiftdatax_train, tsiftdatay_train)
rf2 = RandomForestClassifier(random_state=1)
rf2.fit(tsiftdatax_train, tsiftdatay_train)
#合并dt
label=np.unique(tsiftdatay_train)

probtf = dt.predict_proba(testfeature)
#用概率生成预测
idxtf = np.argmax(probtf, axis=1)
protf = np.amax(probtf, axis=1)


probsift = dt2.predict_proba(tsiftdatax_test)
#用概率生成预测
idxsift = np.argmax(probsift, axis=1)
prosift = np.amax(probsift, axis=1)
predictsift = label[idxsift]

#结合生成新的预测，考虑到predict_prob不如predict效果好，不同时我们只采用TF-IDF模型的predict值
predictcombine=np.zeros(idxsift.shape[0])
for i in range(idxsift.shape[0]):
    #出现不一致，只考虑TF-IDF
    if(label[idxtf[i]]!=dt.predict(testfeature)[i] or label[idxsift[i]]!=dt2.predict(tsiftdatax_test)[i]):
        predictcombine[i]=dt.predict(testfeature)[i]
    else:
        if(protf[i]>prosift[i]):
            predictcombine[i]=label[idxtf[i]]
        else:
            predictcombine[i]=label[idxsift[i]]
#打印准确率，测试集相同
print(f"Combining TF-IDF&SIFT DecisionTree Accuracy score: {accuracy_score(tsiftdatay_test,predictcombine) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT DecisionTree Precision score: {precision_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT DecisionTree Recall score: {recall_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT DecisionTree F1 score: {f1_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
#合并rf
label=np.unique(tsiftdatay_train)

probtf = rf.predict_proba(testfeature)
#用概率生成预测
idxtf = np.argmax(probtf, axis=1)
protf = np.amax(probtf, axis=1)


probsift = rf2.predict_proba(tsiftdatax_test)
#用概率生成预测
idxsift = np.argmax(probsift, axis=1)
prosift = np.amax(probsift, axis=1)
predictsift = label[idxsift]

#结合生成新的预测，考虑到predict_prob不如predict效果好，不同时我们只采用TF-IDF模型的predict值
predictcombine=np.zeros(idxsift.shape[0])
for i in range(idxsift.shape[0]):
    #出现不一致，只考虑TF-IDF
    if(label[idxtf[i]]!=rf.predict(testfeature)[i] or label[idxsift[i]]!=rf2.predict(tsiftdatax_test)[i]):
        predictcombine[i]=rf.predict(testfeature)[i]
    else:
        if(protf[i]>prosift[i]):
            predictcombine[i]=label[idxtf[i]]
        else:
            predictcombine[i]=label[idxsift[i]]
#打印准确率，测试集相同
print(f"Combining TF-IDF&SIFT RandomForest Accuracy score: {accuracy_score(tsiftdatay_test,predictcombine)* 100:.3f}%", )
print(f"Combining TF-IDF&SIFT RandomForest Precision score: {precision_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT RandomForest Recall score: {recall_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
print(f"Combining TF-IDF&SIFT RandomForest F1 score: {f1_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )

可见合并后的随机森林准确率有质的提升，体现出文本图像信息综合利用的优点。这也是我们最终展示的模型之一。

In [ ]:
#adaboost合并

for l in [0.01,0.1,0.3,0.5,1]:
    print("learning rate="+str(l))
    #TF-IDF
    ab = AdaBoostClassifier(learning_rate=l,base_estimator=DecisionTreeClassifier(),random_state=1)
    ab.fit(trainfeature, ttexty_train)

    #SIFT
    ab2 = AdaBoostClassifier(learning_rate=l,base_estimator=DecisionTreeClassifier(),random_state=1)
    ab2.fit(tsiftdatax_train, tsiftdatay_train)

    #合并ab
    label=np.unique(tsiftdatay_train)

    probtf = ab.predict_proba(testfeature)
    #用概率生成预测
    idxtf = np.argmax(probtf, axis=1)
    protf = np.amax(probtf, axis=1)


    probsift = ab2.predict_proba(tsiftdatax_test)
    #用概率生成预测
    idxsift = np.argmax(probsift, axis=1)
    prosift = np.amax(probsift, axis=1)
    predictsift = label[idxsift]

    #结合生成新的预测，考虑到predict_prob不如predict效果好，不同时我们只采用TF-IDF模型的predict值
    predictcombine=np.zeros(idxsift.shape[0])
    for i in range(idxsift.shape[0]):
        #出现不一致，只考虑TF-IDF
        if(label[idxtf[i]]!=ab.predict(testfeature)[i] or label[idxsift[i]]!=ab2.predict(tsiftdatax_test)[i]):
            predictcombine[i]=ab.predict(testfeature)[i]
        else:
            if(protf[i]>prosift[i]):
                predictcombine[i]=label[idxtf[i]]
            else:
                predictcombine[i]=label[idxsift[i]]
    #打印准确率，测试集相同
    print(f"Combining TF-IDF&SIFT AdaBoost Accuracy score: {accuracy_score(tsiftdatay_test,predictcombine) * 100:.3f}%", )
    print(f"Combining TF-IDF&SIFT AdaBoost Precision score: {precision_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
    print(f"Combining TF-IDF&SIFT AdaBoost Recall score: {recall_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )
    print(f"Combining TF-IDF&SIFT AdaBoost F1 score: {f1_score(tsiftdatay_test,predictcombine,average='macro',zero_division=0) * 100:.3f}%", )


此时Adaboost的学习率对其无影响

综上，我们结合文本特征TF-IDF和图像处理SIFT的不同模型整合出了新的分类器，希望综合文本和图像分类的优点给出更好的判断。准确率确实有较大提升，尤其是随机森林，效果已经可以应用。更多文本图像综合处理的方法还有待进一步研究。